In [5]:
from pathlib import Path
import pandas as pd

# === 1) Configura la ruta de la carpeta ===
base = Path(r"C:\Users\Jose Ruiz\Desktop\tripadvisor_output\Hoteles")  
archivos = list(base.glob("reviews_*.csv"))

dfs = []
for f in archivos:
    # === 2) Extraer la parte después del primer '-' ===
    stem = f.stem  
    # Eliminar prefijo 'reviews_' si existe
    name_wo_prefix = stem[len("reviews_"):] if stem.startswith("reviews_") else stem
    # Partir en 2 por el primer '-'
    partes = name_wo_prefix.split("-", 1)
    ciudad_val = partes[1] if len(partes) == 2 else ""   # toda la parte final, p.e. "Los_roques_national_park_coastal_island_insular_region"

    # === 3) Leer CSV con tolerancia de encoding ===
    try:
        df = pd.read_csv(f, encoding="utf-8")
    except UnicodeDecodeError:
        df = pd.read_csv(f, encoding="latin-1")

    # === 4) Renombrar Title -> Lugar si existe ===
    if "Title" in df.columns:
        df = df.rename(columns={"Title": "Lugar"})

    # === 5) Añadir columnas ===
    df["Ciudad"] = ciudad_val              
    df["Tipo"] = "hotel"

    dfs.append(df)

# === 6) Unir y exportar ===
if dfs:
    final = pd.concat(dfs, ignore_index=True)
    salida = base / "hoteles_venezuela_tripadvisor.csv"
    final.to_csv(salida, index=False, encoding="utf-8-sig")
    print(f"Guardado: {salida} | Filas: {len(final)} | Archivos procesados: {len(archivos)}")
else:
    print("No se encontraron archivos 'reviews_*.csv' en la carpeta.")

Guardado: C:\Users\Jose Ruiz\Desktop\tripadvisor_output\Hoteles\hoteles_venezuela_tripadvisor.csv | Filas: 41950 | Archivos procesados: 100
